# Advanced Calibration: Bayesian MCMC with Uncertainty Quantification

This notebook demonstrates **Bayesian calibration** using Markov Chain Monte Carlo (MCMC) methods. Unlike the neural network approach (which provides point estimates), Bayesian methods give us **full posterior distributions** and **uncertainty quantification**.

## Why Bayesian Calibration?

- **Uncertainty quantification**: Get credible intervals, not just point estimates
- **Incorporate prior knowledge**: Use domain expertise as priors
- **Model comparison**: Compute Bayes factors to choose between models
- **Robust to noise**: Naturally handles noisy market data

## Trade-offs

- ⏱️ **Slower**: ~1-5 minutes vs ~10ms for neural network
- 📊 **More informative**: Full distributions vs point estimates
- 🎯 **Best for**: Critical calibrations, research, model validation

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and TensorFlow Probability
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

# Import calibration modules
from models.pricing_engine.levy_models import variance_gamma_char_func
from models.pricing_engine.fourier_pricer import price_surface
from models.bayesian_calibration.mcmc import BayesianCalibrator
from models.bayesian_calibration.diagnostics import compute_rhat, compute_ess
from models.bayesian_calibration.uncertainty_propagation import propagate_uncertainty_single_option

# Plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ Imports successful")
print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Probability version: {tfp.__version__}")

## 2. Generate Synthetic Market Data

We'll create a "ground truth" option surface with known parameters, then add market noise.

In [ ]:
# True parameters (our ground truth)
sigma_true = 0.25
nu_true = 0.35
theta_true = -0.15

# Market parameters
S0 = 100.0
r = 0.05

# Define grid (smaller for faster MCMC)
strikes = np.array([90, 95, 100, 105, 110])
maturities = np.array([0.25, 0.5, 1.0])

# Generate true surface
char_func_true = variance_gamma_char_func(sigma_true, nu_true, theta_true)
true_surface = price_surface(S0, strikes, maturities, r, char_func_true)

# Add realistic market noise (bid-ask spread ~0.5%)
np.random.seed(42)
noise = np.random.normal(0, 0.005 * true_surface)
observed_surface = true_surface + noise

print("Ground Truth Parameters:")
print(f"  σ (sigma) = {sigma_true}")
print(f"  ν (nu)    = {nu_true}")
print(f"  θ (theta) = {theta_true}")
print(f"\nOption surface shape: {observed_surface.shape}")
print(f"Price range: ${observed_surface.min():.2f} - ${observed_surface.max():.2f}")

## 3. Visualize Observed vs True Prices

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for i, T in enumerate(maturities):
    ax = axes[i]
    ax.plot(strikes, true_surface[:, i], 'o-', label='True', linewidth=2, markersize=8)
    ax.plot(strikes, observed_surface[:, i], 's--', label='Observed (noisy)', 
            linewidth=2, markersize=6, alpha=0.7)
    ax.set_xlabel('Strike', fontsize=11)
    ax.set_ylabel('Option Price', fontsize=11)
    ax.set_title(f'T = {T} years', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Define Prior Distributions

We encode our prior beliefs about the parameters:

- **σ (sigma)**: Volatility is typically 0.1-0.5 → LogNormal prior
- **ν (nu)**: Variance rate is positive, mean ~0.3 → Gamma prior  
- **θ (theta)**: Drift is typically negative for equities → Normal prior

In [ ]:
# Initialize Bayesian calibrator
calibrator = BayesianCalibrator(model_name='VarianceGamma')

# Build prior
prior = calibrator.build_prior()

# Sample from prior to visualize
prior_samples = prior.sample(5000)

# Plot prior distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
param_names = ['sigma', 'nu', 'theta']

for i, (name, ax) in enumerate(zip(param_names, axes)):
    samples = prior_samples[name].numpy()
    ax.hist(samples, bins=50, density=True, alpha=0.7, edgecolor='black')
    ax.axvline([sigma_true, nu_true, theta_true][i], color='red', 
                linestyle='--', linewidth=2, label='True value')
    ax.set_xlabel(f'{name}', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'Prior: {name}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Prior distributions defined")

## 5. Run MCMC Sampling (NUTS)

We'll use the **No-U-Turn Sampler (NUTS)**, a state-of-the-art Hamiltonian Monte Carlo algorithm.

⚠️ **Note**: This cell may take 2-5 minutes to run.

In [ ]:
import time

print("Running MCMC sampling...")
print("This may take 2-5 minutes. Please wait...\n")

start_time = time.time()

# Run MCMC
posterior_samples, diagnostics = calibrator.run_mcmc(
    observed_prices=observed_surface.flatten(),
    strikes=strikes,
    maturities=maturities,
    S0=S0,
    r=r,
    num_samples=2000,      # Number of samples per chain
    num_burnin=1000,       # Burn-in samples to discard
    num_chains=4,          # Number of parallel chains
    step_size=0.01         # Initial step size
)

end_time = time.time()
sampling_time = end_time - start_time

print(f"\n✓ MCMC sampling complete!")
print(f"  Total time: {sampling_time:.1f} seconds")
print(f"  Samples per chain: {posterior_samples['sigma'].shape[0]}")
print(f"  Number of chains: {posterior_samples['sigma'].shape[1]}")

## 6. Convergence Diagnostics

Before trusting the results, we must verify MCMC convergence:

- **R-hat**: Should be < 1.01 (measures convergence across chains)
- **ESS**: Effective Sample Size (should be > 400 for reliable estimates)

In [ ]:
print("="*60)
print("CONVERGENCE DIAGNOSTICS")
print("="*60)

print(f"\n{'Parameter':<10} {'R-hat':<12} {'ESS':<12} {'Status'}")
print("-"*60)

for param in ['sigma', 'nu', 'theta']:
    chains = posterior_samples[param]
    rhat = compute_rhat(chains)
    ess = compute_ess(chains)
    
    status = "✓ Good" if (rhat < 1.01 and ess > 400) else "⚠ Check"
    print(f"{param:<10} {rhat:<12.4f} {ess:<12.0f} {status}")

print("="*60)
print("\nInterpretation:")
print("  R-hat < 1.01: Chains have converged ✓")
print("  ESS > 400: Sufficient effective samples for inference ✓")

## 7. Trace Plots

Visual inspection of chain mixing and convergence.

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

param_names = ['sigma', 'nu', 'theta']
true_values = [sigma_true, nu_true, theta_true]

for i, (param, true_val) in enumerate(zip(param_names, true_values)):
    chains = posterior_samples[param]
    
    # Left: Trace plot
    ax_trace = axes[i, 0]
    for chain_idx in range(chains.shape[1]):
        ax_trace.plot(chains[:, chain_idx], alpha=0.6, linewidth=0.8)
    ax_trace.axhline(true_val, color='red', linestyle='--', linewidth=2, label='True')
    ax_trace.set_ylabel(param, fontsize=11)
    ax_trace.set_xlabel('Iteration', fontsize=10)
    ax_trace.set_title(f'Trace: {param}', fontsize=11, fontweight='bold')
    ax_trace.legend()
    ax_trace.grid(alpha=0.3)
    
    # Right: Autocorrelation
    ax_acf = axes[i, 1]
    samples_flat = chains.flatten()
    from pandas.plotting import autocorrelation_plot
    autocorrelation_plot(pd.Series(samples_flat), ax=ax_acf, color='steelblue')
    ax_acf.set_title(f'Autocorrelation: {param}', fontsize=11, fontweight='bold')
    ax_acf.set_xlim(0, 100)
    ax_acf.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Good mixing: chains overlap and explore parameter space")
print("Low autocorrelation: samples are nearly independent")

## 8. Posterior Distributions

The main result: **posterior distributions** for each parameter.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (param, true_val, ax) in enumerate(zip(param_names, true_values, axes)):
    samples_flat = posterior_samples[param].flatten()
    
    # Histogram
    ax.hist(samples_flat, bins=50, density=True, alpha=0.7, 
            edgecolor='black', color='steelblue')
    
    # True value
    ax.axvline(true_val, color='red', linestyle='--', linewidth=2.5, label='True value')
    
    # Posterior mean
    posterior_mean = np.mean(samples_flat)
    ax.axvline(posterior_mean, color='green', linestyle='-', linewidth=2.5, label='Posterior mean')
    
    # 95% HDI (Highest Density Interval)
    hdi_lower = np.percentile(samples_flat, 2.5)
    hdi_upper = np.percentile(samples_flat, 97.5)
    ax.axvspan(hdi_lower, hdi_upper, alpha=0.2, color='green', label='95% HDI')
    
    ax.set_xlabel(param, fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'Posterior: {param}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("="*70)
print("POSTERIOR SUMMARY")
print("="*70)
print(f"\n{'Parameter':<10} {'True':<10} {'Mean':<10} {'Std':<10} {'95% HDI'}")
print("-"*70)

for param, true_val in zip(param_names, true_values):
    samples_flat = posterior_samples[param].flatten()
    mean = np.mean(samples_flat)
    std = np.std(samples_flat)
    hdi_lower = np.percentile(samples_flat, 2.5)
    hdi_upper = np.percentile(samples_flat, 97.5)
    
    print(f"{param:<10} {true_val:<10.4f} {mean:<10.4f} {std:<10.4f} [{hdi_lower:.4f}, {hdi_upper:.4f}]")

print("="*70)

## 9. Uncertainty Propagation to Option Pricing

We can propagate parameter uncertainty to get **prediction intervals** for option prices.

In [ ]:
# Select a specific option to analyze
K_test = 100.0
T_test = 1.0

# Propagate uncertainty
result = propagate_uncertainty_single_option(
    posterior_samples=posterior_samples,
    model_name='VarianceGamma',
    S0=S0,
    K=K_test,
    T=T_test,
    r=r,
    num_samples=1000
)

print("="*60)
print(f"OPTION PRICING WITH UNCERTAINTY (K={K_test}, T={T_test})")
print("="*60)
print(f"\nPredicted price: ${result['mean']:.4f}")
print(f"Standard deviation: ${result['std']:.4f}")
print(f"95% Credible interval: [${result['hdi_95_lower']:.4f}, ${result['hdi_95_upper']:.4f}]")
print("="*60)

# Plot price distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(result['price_samples'], bins=50, density=True, alpha=0.7, 
        edgecolor='black', color='steelblue')
ax.axvline(result['mean'], color='green', linestyle='-', linewidth=2.5, label='Mean')
ax.axvspan(result['hdi_95_lower'], result['hdi_95_upper'], 
           alpha=0.2, color='green', label='95% HDI')
ax.set_xlabel('Option Price', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title(f'Predictive Distribution for Option (K={K_test}, T={T_test})', 
             fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Parameter Correlations (Corner Plot)

Visualize joint posterior distributions and parameter correlations.

In [ ]:
# Prepare data for corner plot
samples_array = np.column_stack([
    posterior_samples['sigma'].flatten(),
    posterior_samples['nu'].flatten(),
    posterior_samples['theta'].flatten()
])

# Create corner plot using seaborn pairplot
df_samples = pd.DataFrame(samples_array, columns=['sigma', 'nu', 'theta'])
g = sns.pairplot(df_samples, diag_kind='hist', plot_kws={'alpha': 0.3, 's': 5}, 
                 diag_kws={'bins': 40, 'edgecolor': 'black'})
g.fig.suptitle('Posterior Parameter Correlations', y=1.02, fontsize=14, fontweight='bold')

# Add true values
for i, param in enumerate(['sigma', 'nu', 'theta']):
    for j in range(3):
        if i != j:
            g.axes[i, j].axhline(true_values[i], color='red', linestyle='--', alpha=0.7)
            g.axes[i, j].axvline(true_values[j], color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Compute correlation matrix
corr_matrix = df_samples.corr()
print("\nPosterior Correlation Matrix:")
print(corr_matrix.round(3))

## Summary

In this notebook, we:

1. ✅ Generated synthetic market data with realistic noise
2. ✅ Defined informative prior distributions for VG parameters
3. ✅ Ran MCMC sampling using the NUTS algorithm
4. ✅ Verified convergence using R-hat and ESS diagnostics
5. ✅ Visualized posterior distributions and parameter correlations
6. ✅ Propagated uncertainty to option pricing predictions
7. ✅ Obtained **credible intervals** for all parameters and predictions

## Key Insights

- **Bayesian methods provide full uncertainty quantification**
- **MCMC is slower (~minutes) but more informative than neural networks (~ms)**
- **Best use case**: Critical calibrations where uncertainty matters (e.g., risk management, regulatory reporting)
- **Hybrid approach**: Use neural networks for speed, Bayesian for validation

## Next Steps

- Try CGMY model calibration
- Experiment with different priors
- Compare Bayesian vs ML calibration on real market data
- Implement model comparison using WAIC or Bayes factors